In [3]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numba import njit
import itertools
from tqdm.auto import tqdm
from alg.functions import functions
from alg.bits import find_int_number_of_bits, bit_metrics, \
                        bits_to_value, value_to_bits, \
                        bitstring_to_array, array_to_bitstring
from alg.sa import sa, acc_prob_boltz, acc_prob_quench, acc_prob_fast, acc_prob_ln
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 20

# https://matplotlib.org/stable/gallery/style_sheets/grayscale.html
# https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_prop_cycle.html
plt.style.use('grayscale')

tqdm.pandas()

n_runs = 30


ap_names = ['boltz', 'quench', 'fast', 'ln']
ap_funcs = [acc_prob_boltz, acc_prob_quench, acc_prob_fast, acc_prob_ln]

root = os.path.join('experiments', 'experiment_2_classic_methods')

In [4]:
tolerance = 1e-5
for function in functions:
    n_bits_value, solution_powers, max_value_on_bits = bit_metrics(function.lb, function.ub, tolerance)
    print(function.name, n_bits_value, function.lb, function.ub)

Sphere 20 -5.12 5.12
Bohachevsky 25 -100 100
Rosenbrock 19 -2.048 2.048
Rastrigin 20 -5.12 5.12
Ackley 20 -5.12 5.12
Griewangk 27 -600 600
Schwefel 27 -500 500
Eggholder 27 -512 512
Rana 27 -512 512


In [3]:
# %timeit sa(target_function=functions[0].f, lb=functions[0].lb, ub=functions[0].ub, \
#                            acc_prob_=acc_prob_quench, T0=500, apf_param=0.01, p_max=1, \
#                            n_dims=2, max_iter=max_iter)

In [5]:
def generate_random_transitions(function, n_dims, tolerance=1e-5, n_reps=1e5):
    lb = function.lb
    ub = function.ub

    n_bits_value, solution_powers, max_value_on_bits = bit_metrics(lb, ub, tolerance)
    n_bits = n_dims * n_bits_value

    x = (np.random.uniform(0, 1, n_bits) < 0.5).astype(np.int32)
    xa = bitstring_to_array(x, n_dims, n_bits_value, lb, ub, max_value_on_bits, solution_powers)
    score = function.f(xa)

    _scores = [score]
    for _ in range(n_reps):
        mutation_bit = np.random.randint(0, n_bits)
        x[mutation_bit] = 1 - x[mutation_bit]
        xa = bitstring_to_array(x, n_dims, n_bits_value, lb, ub, max_value_on_bits, solution_powers)
        score = function.f(xa)
        _scores.append(score)

    _scores = np.array(_scores)

    maxs = np.maximum.accumulate(_scores)
    mins = np.minimum.accumulate(_scores)
    return mins, maxs

def g(T, mins, maxs):
    return np.exp(-maxs/T).sum() / np.exp(-mins/T).sum()

def compute_T0(mins, maxs, p_desired=0.8, p=1.1, eps=1e-3):
    T  = 100000
    n = 1
    
    xi = g(T, mins, maxs)

    while abs(xi - p_desired) > eps:
        T = T * ((np.log(xi) / np.log(p_desired)) ** (1 / p_desired))
        n += 1
        xi = g(T, mins, maxs)

    return T


loop = list(itertools.product(range(len(functions)), range(2, 21)))

init_T0 = list()
for func_index, n_dims in tqdm(loop, leave=False):
    function = functions[func_index]
    mins, maxs = generate_random_transitions(function, n_dims, tolerance=1e-5, n_reps=int(1e5))
    T0 = compute_T0(mins, maxs, p_desired=0.8)
    init_T0.append({'func_index':func_index, 'func_name':function.name, 'n_dims':n_dims, 'T0':T0})
init_T0 = pd.DataFrame(init_T0).set_index(['func_name', 'n_dims'])

  0%|          | 0/171 [00:00<?, ?it/s]

In [17]:
fnames = [f.name for f in functions]

In [31]:
latex_filename = 'init_T0_table.tex'
latex_T0 = init_T0.T0.reset_index().rename(columns={'func_name':'Func', 'n_dims':'Dim'}) \
        .pivot(columns='Func', index='Dim', values='T0')[fnames]
latex_T0.to_latex(latex_filename, float_format="%.3f")

C:\Users\dinus\AppData\Local\Temp\ipykernel_3928\3846000821.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_T0.round(2).to_latex(latex_filename, float_format="%.2f")


In [6]:
acc_prob = ap_funcs[1]
target_function = functions[0]
n_dims = 20
b = 0.01
iter_factor = 500
max_iter = int(iter_factor * (n_dims ** 2))
T0 = init_T0.loc[(target_function.name, n_dims)].T0

scores, aps, outcomes = sa(target_function=target_function.f, \
                           lb=target_function.lb, ub=target_function.ub, \
                           acc_prob_=acc_prob, T0=T0, apf_param=b, p_max=0.8, \
                           n_dims=n_dims, max_iter=max_iter)

In [7]:
def run_experiment_2_1(n_runs=30):
    result_root = os.path.join(root, 'classic_sa')
    os.makedirs(result_root, exist_ok=True)
    apf_param = 1e-4 # will not be used. Required by sa call.
    iter_factor = 500
    
    for run_ix in tqdm(range(n_runs), total=n_runs, leave=False):
        result_file = os.path.join(result_root, f'run={run_ix}.parquet')
        if not os.path.exists(result_file):
            run_results = []
            for ap_ix, acc_prob in tqdm(enumerate(ap_funcs), total=len(ap_funcs), leave=False):
                for target_function in tqdm(functions, total=len(functions), leave=False):
                    for n_dims in np.arange(2, 21)[::-1]:
                        max_iter = int(iter_factor * (n_dims ** 2))
                        T0 = init_T0.loc[(target_function.name, n_dims)].T0
                        n_dims = int(n_dims) # int32 vs int64 numba error

                        scores, aps, outcomes = sa(target_function=target_function.f, \
                           lb=target_function.lb, ub=target_function.ub, \
                           acc_prob_=acc_prob, T0=T0, apf_param=apf_param, p_max=0.8, \
                           n_dims=n_dims, max_iter=max_iter, tolerance=tolerance)
                        
                        run_results.append({
                            'ap':ap_names[ap_ix],
                            'target':target_function.name,
                            'n_dims':n_dims,
                            'min_score':scores.min()
                        })
            _df = pd.DataFrame(run_results)
            _df.to_parquet(result_file)

# run_experiment_2_1(n_runs=n_runs)

- Outcomes over optimization moment.
- These cover the early stopping scenario.

In [8]:
def run_experiment_2_2(n_runs=30, tolerance=1e-5):
    result_root = os.path.join(root, 'outcomes')
    os.makedirs(result_root, exist_ok=True)
    p_max = 0.8
    iter_factor = 500
    n_dims = 2
    
    for run_ix in tqdm(range(n_runs), total=n_runs, leave=False):
        for ap_ix, acc_prob in tqdm(enumerate(ap_funcs), total=len(ap_funcs), leave=False):
            for target_function in tqdm(functions, total=len(functions), leave=False):
                for n_dims in [2, 10]: # 2, 11, 20
                    result_file = os.path.join(result_root,\
                            f'run={run_ix}_ap={ap_names[ap_ix]}_target={target_function.name}_d={n_dims}.parquet')
                    if not os.path.exists(result_file):
                        max_iter = int(iter_factor * (n_dims ** 2))
                        scores, aps, outcomes = sa(target_function=target_function.f, \
                               lb=target_function.lb, ub=target_function.ub, \
                               acc_prob_=acc_prob, T0=T0, apf_param=1e-4, p_max=p_max, \
                               n_dims=int(n_dims), max_iter=max_iter, tolerance=tolerance)       
                        _df = pd.DataFrame([scores, aps, outcomes]).T
                        _df.columns = ['scores', 'accept_probs', 'outcomes']
    #                     _df['ap'] = ap_names[ap_ix]
    #                     _df['target'] = target_function.name
                        _df.to_parquet(result_file)

run_experiment_2_2(n_runs=n_runs)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
def run_experiment_2_3(n_runs=30, budget=50):
    result_root = os.path.join(root, f'budget_{budget}')
    os.makedirs(result_root, exist_ok=True)
    apf_param = 1e-4
    iter_factor = budget
    if budget == 50:
        possible_dims = [20]
    else:
        possible_dims = [10]
#     possible_dims = [2, 5, 10]
    for run_ix in tqdm(range(n_runs), total=n_runs, leave=False):
        result_file = os.path.join(result_root, f'run={run_ix}.parquet')
        if not os.path.exists(result_file):
            run_results = []
            for ap_ix, acc_prob in tqdm(enumerate(ap_funcs), total=len(ap_funcs), leave=False):
                for target_function in tqdm(functions, total=len(functions), leave=False):
                    for n_dims in possible_dims:
                        max_iter = int(iter_factor * (n_dims ** 2))
                        T0 = init_T0.loc[(target_function.name, n_dims)].T0
                        n_dims = int(n_dims) # int32 vs int64 numba error

                        scores, aps, outcomes = sa(target_function=target_function.f, \
                           lb=target_function.lb, ub=target_function.ub, \
                           acc_prob_=acc_prob, T0=T0, apf_param=apf_param, p_max=0.8, \
                           n_dims=n_dims, max_iter=max_iter, tolerance=tolerance)
                        
                        run_results.append({
                            'ap':ap_names[ap_ix],
                            'target':target_function.name,
                            'n_dims':n_dims,
                            'min_score':scores.min()
                        })
            _df = pd.DataFrame(run_results)
            _df.to_parquet(result_file)

run_experiment_2_3(n_runs=n_runs, budget=50)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
run_experiment_2_3(n_runs=n_runs, budget=5000)

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]